In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit
import ta

In [2]:
# Data
symbol = 'SPXL'  # Using QQQ as an example

strat_name = 'dual_ema_crossover'

freq='1D'

price = vbt.YFData.download(
    symbol,
    start='1/1/2023',
    interval='1d'
).get('Close')

fast_level = np.arange(9,27)
slow_level = np.arange(26,60)
EMA = vbt.IndicatorFactory.from_ta('EMAIndicator')
# ema13 = EMA.run(price, 5)
# ema48 = EMA.run(price, 10)

windows = np.arange(2, 101)
ema13, ema48 = EMA.run_combs(price, window=windows, r=2, short_names=['ema_fast', 'ema_slow'])

entries = ema13.ema_indicator_crossed_above(ema48)
exits = ema48.ema_indicator_crossed_below(ema48)

@njit
def adjust_sl_func_nb(c):
    current_profit = (c.val_price_now - c.init_price) / c.init_price
    if current_profit >= 0.6:
        return 0.4, True
    if current_profit >= 0.40:
        return 0.05, True
    elif current_profit >= 0.2:
        return 0.01, True
    # elif current_profit >= 0.05:
    #     return 0.01, True
    # else:
    #     return 0.2, False
    return c.curr_stop, c.curr_trail

take_profit_pct = 20  # 10%
stop_loss_pct = 0.1  # -5%

# Backtest with vectorbt, specifying short entries and exits
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    short_entries=exits,
    short_exits=entries,
    # init_cash=10000,
    # size=1,
    # size_type=vbt.portfolio.enums.SizeType.Percent,
    sl_stop=stop_loss_pct,
    adjust_sl_func_nb=adjust_sl_func_nb,
    tp_stop=take_profit_pct,
    freq=freq
)

In [3]:
pf.total_return()[::-1]

ema_fast_window  ema_slow_window
99               100                0.000000
98               100                0.000000
                 99                 0.000000
97               100                0.000000
                 99                 0.000000
                                      ...   
2                7                  0.577341
                 6                  0.496613
                 5                  0.456876
                 4                  0.503389
                 3                  0.705587
Name: total_return, Length: 4851, dtype: float64

In [4]:
xx = pf.get_orders()

# pf.cumulative_returns().vbt.plot().show()
max_pf = pf.total_return().idxmax()


In [5]:
max_pf

(2, 3)

In [6]:
feats = list(pf.total_return().index.names)

In [7]:
dict(m=1)

{'m': 1}

In [8]:
best_feats = dict({f:m for f,m in zip(feats, max_pf)})

In [9]:
best_feats

{'ema_fast_window': 2, 'ema_slow_window': 3}

In [10]:

opf = pf[max_pf]
opf.plot().show()

# Show performance metrics
print(max_pf)


print(opf.stats())


(2, 3)
Start                         2023-01-03 05:00:00+00:00
End                           2024-04-12 04:00:00+00:00
Period                                321 days 00:00:00
Start Value                                       100.0
End Value                                    170.558721
Total Return [%]                              70.558721
Benchmark Return [%]                         104.938796
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              32.791902
Max Drawdown Duration                 119 days 00:00:00
Total Trades                                          9
Total Closed Trades                                   8
Total Open Trades                                     1
Open Trade PnL                               -14.103055
Win Rate [%]                                       62.5
Best Trade [%]                                30.955186
Worst Trade [%]                          

In [11]:
# Make dataframe to store 👉🏽 `strategy_stats`
base_stats = opf.stats()

In [12]:
base_stats['Features'] = best_feats
base_stats['Ticker'] = symbol

In [13]:
df_store = pd.DataFrame(base_stats).T.reset_index(drop=True).convert_dtypes()

In [14]:
import re

def to_snake_case(df):
    """
    Convert the column names of a DataFrame to lowercase snake_case format,
    removing special characters and newlines before conversion.
    
    Parameters:
    - df: pandas.DataFrame
    
    Returns:
    - DataFrame with column names in snake_case format.
    """
    # First, replace newlines with spaces, then remove special characters except spaces
    df.columns = [col.replace('\n', '') for col in df.columns]
    df.columns = [col.replace('[%]', '') for col in df.columns]
    df.columns = [col.strip() for col in df.columns]
    df.columns = [col.replace(' ', '_') for col in df.columns]
    df.columns = [col.lower() for col in df.columns]
    
    return df

In [15]:
from data_process.store_strategy import ProcessData

In [16]:
p = ProcessData()

In [17]:
p.store_df(df_store)

"An error occurred: Object of type dict in column 'Features' is not JSON serializable"

In [18]:
XXX

NameError: name 'XXX' is not defined

In [ ]:
df_store

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,Features,Ticker
0,2023-01-03 05:00:00+00:00,2024-04-12 04:00:00+00:00,321 days,100,170.558663,70.558663,104.938783,100,0,32.791902,...,37 days 14:24:00,30 days 08:00:00,2.845572,10.582714,1.623011,2.546631,1.260262,2.434286,"{'ema_fast_window': 2, 'ema_slow_window': 3}",SPXL


In [ ]:
to_snake_case(df_store)

,start,end,period,start_value,end_value,total_return,benchmark_return,max_gross_exposure,total_fees_paid,max_drawdown,...,avg_winning_trade_duration,avg_losing_trade_duration,profit_factor,expectancy,sharpe_ratio,calmar_ratio,omega_ratio,sortino_ratio,features,ticker
0,2023-01-03 05:00:00+00:00,2024-04-12 04:00:00+00:00,321 days,100,170.558663,70.558663,104.938783,100,0,32.791902,...,37 days 14:24:00,30 days 08:00:00,2.845572,10.582714,1.623011,2.546631,1.260262,2.434286,"{'ema_fast_window': 2, 'ema_slow_window': 3}",SPXL


In [ ]:
df_store.to_csv('strategy_stats.csv')

In [ ]:
df_store

,start,end,period,start_value,end_value,total_return,benchmark_return,max_gross_exposure,total_fees_paid,max_drawdown,...,avg_winning_trade_duration,avg_losing_trade_duration,profit_factor,expectancy,sharpe_ratio,calmar_ratio,omega_ratio,sortino_ratio,features,ticker
0,2023-01-03 05:00:00+00:00,2024-04-12 04:00:00+00:00,321 days,100,170.558663,70.558663,104.938783,100,0,32.791902,...,37 days 14:24:00,30 days 08:00:00,2.845572,10.582714,1.623011,2.546631,1.260262,2.434286,"{'ema_fast_window': 2, 'ema_slow_window': 3}",SPXL


In [ ]:
xx.config['close'][max_pf]

Date
2023-01-03 05:00:00+00:00     60.427799
2023-01-04 05:00:00+00:00     61.757282
2023-01-05 05:00:00+00:00     59.610405
2023-01-06 05:00:00+00:00     63.657970
2023-01-09 05:00:00+00:00     63.579178
                                ...    
2024-04-08 04:00:00+00:00    130.070007
2024-04-09 04:00:00+00:00    130.520004
2024-04-10 04:00:00+00:00    126.610001
2024-04-11 04:00:00+00:00    129.399994
2024-04-12 04:00:00+00:00    123.839996
Name: (2, 3), Length: 321, dtype: float64

In [ ]:
pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')

fig = pf.total_return().vbt.heatmap(
    x_level='fast_ema', y_level='slow_ema', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

ValueError: 'fast_ema' is not in list